In [1]:
import pandas as pd
import numpy as np

In [2]:
entprise_train = pd.read_csv('train/entprise_info.csv',engine='python')
base_info = pd.read_csv('train/base_info.csv',engine='python',encoding='utf-8')
entprise_test = pd.read_csv('entprise_evaluate.csv')

In [3]:
entprise = pd.concat([entprise_train,entprise_test])
data = pd.merge(entprise,base_info,on='id')
data.drop(['ptbusscope','midpreindcode','score'],axis=1,inplace=True)

In [4]:
data['opform'].fillna(10,inplace=True)
data['opform'] = data['opform'].astype(str)

In [5]:
from sklearn.preprocessing import LabelEncoder
labelencoder1 = LabelEncoder()
labelencoder2 = LabelEncoder()
labelencoder3 = LabelEncoder()
labelencoder4 = LabelEncoder()
data['opform'] = labelencoder1.fit_transform(data['opform'])
data['oploc'] = labelencoder2.fit_transform(data['oploc'])
data['industryphy'] = labelencoder3.fit_transform(data['industryphy'])
data['oplocdistrict'] = labelencoder4.fit_transform(data['oplocdistrict'])

In [6]:
def opto_process(x):
    if len(x)>3:
        return x[:4]
    else:
        return x
data['opfrom'] = data['opfrom'].map(lambda x:x[:4])
data['opto'].fillna('nan',inplace=True)
data['opto'] = data['opto'].map(opto_process)
data['opfrom'] = data['opfrom'].astype(int)
data['opto'] = data['opto'].astype(float)

In [25]:
data['enttypeitem'] = data['enttypegb']//10*10
data['enttypeminu'] = data['enttypegb']
data.loc[data['opto'].isnull(),"opto"] = data[data['opto'].isnull()]['opfrom']+50.0
data['opscope'] = data['opscope'].apply(lambda x:x.replace("、","/").replace("；","/").replace("，","/").replace("。","/"))
data['opscope'] = data['opscope'].apply(lambda x:x.replace("（","/").replace("）","/").replace("，","/").replace("。","/"))
le6 = LabelEncoder()
data['opscope'] = data['opscope'].apply(lambda x:x.split('/')[-2:])
data['opscope_label'] = le6.fit_transform(data['opscope'])
le7 = LabelEncoder()
data['dom'] = le7.fit_transform(data['dom'])
data['compform'] = data['compform'].fillna(-1)
data['industryco'] = data['industryco'].fillna(data['industryco'].mean())

AttributeError: 'list' object has no attribute 'replace'

In [26]:
x_train = data[data['label'].notnull()]
y_train = x_train['label']
x_test = data[data['label'].isnull()]
features_name = [i for i in data.columns if i not in ['id','reccap','regcap','enttypegb','empnum','parnum','exenum','venind','forreccap','forregcap','congro','opscope','label','dom','protype']]
x_train = x_train[features_name]
x_test = x_test[features_name]

In [27]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14865 entries, 0 to 14864
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   oplocdistrict  14865 non-null  int64  
 1   industryphy    14865 non-null  int64  
 2   industryco     14864 non-null  float64
 3   enttype        14865 non-null  int64  
 4   enttypeitem    14865 non-null  int64  
 5   opfrom         14865 non-null  int32  
 6   opto           14865 non-null  float64
 7   state          14865 non-null  int64  
 8   orgid          14865 non-null  int64  
 9   jobid          14865 non-null  int64  
 10  adbusign       14865 non-null  int64  
 11  townsign       14865 non-null  int64  
 12  regtype        14865 non-null  int64  
 13  compform       14865 non-null  float64
 14  opform         14865 non-null  int64  
 15  enttypeminu    14865 non-null  int64  
 16  oploc          14865 non-null  int64  
 17  opscope_label  14865 non-null  int64  
dtypes: flo

In [68]:
from sklearn.metrics import auc, roc_curve,f1_score
from sklearn.model_selection import StratifiedKFold, KFold
#from catboost import CatBoostRegressor
import lightgbm as lgb
# 模型
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=100,
                              learning_rate=0.08, n_estimators=50, max_depth=5, 
                              metric='rmse', bagging_fraction = 0.8, feature_fraction = 0.5)
def stat_func(x):
    l1 = []
    for i in x:
        if i>0.5:
            a = 1
            l1.append(a)
        else:
            a = 0
            l1.append(a)
    return l1
cat_list = [3]
# 本地验证
kf = KFold(n_splits=5,shuffle=True,random_state=2020)
devscore = []
tevscore = []
count = 1
for tidx,didx in kf.split(x_train.index):
    print('************************************ {} ************************************'.format(str(count)))
    count += 1
    tf = x_train.iloc[tidx]
    df = x_train.iloc[didx]
    tt = y_train.iloc[tidx]
    dt = y_train.iloc[didx]
    lgb_train = lgb.Dataset(tf,tt,free_raw_data=False)
    lgb_eval = lgb.Dataset(df,dt,reference=lgb_train,free_raw_data=False)
    model_lgb.fit(tf,tt)
    #catr.fit(tf, tt,cat_features=cat_list)
    pre_train = model_lgb.predict(tf)
    stat_pre_train = stat_func(pre_train)
    score = f1_score(stat_pre_train,tt)
    tevscore.append(score)
    print("train F1：%s"%score)
    pre = model_lgb.predict(df)
    stat_pre = stat_func(pre)
    score = f1_score(stat_pre,dt)
    devscore.append(score)
    print("Valid F1：%s"%score)
print("Train average F1：%s"%np.mean(tevscore),"Valid average F1：%s"%np.mean(devscore))

************************************ 1 ************************************
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
train F1：0.8455384615384616
Valid F1：0.7817258883248731
************************************ 2 ************************************
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
train F1：0.8444444444444444
Valid F1：0.8011527377521614
************************************ 3 ************************************
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.8, subs

In [33]:
from sklearn.metrics import f1_score
def stat_func(x):
    l1 = []
    for i in x:
        if i>0.5:
            a = 1
            l1.append(a)
        else:
            a = 0
            l1.append(a)
    return l1
valid_pre  = stat_func(valid_pre)
print(f1_score(valid_pre,y_valid))

0.7874015748031497
